<a href="https://colab.research.google.com/github/mugalan/working/blob/main/hierarchical_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

* https://arxiv.org/pdf/2303.03293
* https://ris.utwente.nl/ws/files/52018562/Kienreich2012graph.pdf

# Python Class Definition

In [ ]:
import networkx as nx
import graphviz
from IPython.display import display, Image

class HierarchicalGraph:
    def __init__(self, nodes_data, edges_data):
        self.nodes_data = nodes_data
        self.edges_data = edges_data
        self._assign_group_colors()
        self.inner_graph = nx.MultiDiGraph()
        self.outer_graph = nx.DiGraph()
        self.create_hierarchical_graphs_iterative()


    def _assign_group_colors(self):
        """Assign consistent colors to groups."""
        default_colors = [
            'lightblue', 'lightgreen', 'lightyellow', 'lightpink', 'lightgray',
            'lightsalmon', 'lightcyan', 'wheat', 'plum', 'lightgoldenrod'
        ]
        groups = sorted(set(node['group'] for node in self.nodes_data))
        self.group_colors = {
            group: default_colors[i % len(default_colors)]
            for i, group in enumerate(groups)
        }



    def create_hierarchical_graphs_iterative(self):
        inner_graph = nx.MultiDiGraph()

        # --- Add nodes to inner graph ---
        for node_data in self.nodes_data:
            label = node_data['label']
            attributes = {k: v for k, v in node_data.items() if k != 'label'}
            inner_graph.add_node(label, **attributes)

        # --- Add edges to inner graph ---
        for edge_data in self.edges_data:
            start = edge_data['start']
            end = edge_data['end']
            attributes = {k: v for k, v in edge_data.items() if k not in ['start', 'end']}
            inner_graph.add_edge(start, end, **attributes)

        # --- Create outer MultiDiGraph ---
        outer_graph = nx.MultiDiGraph()

        # --- Add group nodes to outer graph ---
        group_nodes = set(d['group'] for d in self.nodes_data)
        outer_graph.add_nodes_from(group_nodes)

        # --- Add inter-group edges to outer graph (one-to-one mapping from inner) ---
        for u, v, data in inner_graph.edges(data=True):
            group_u = inner_graph.nodes[u].get('group')
            group_v = inner_graph.nodes[v].get('group')

            if group_u != group_v:
                # Preserve edge attributes (e.g., type, weight, color)
                outer_graph.add_edge(group_u, group_v, **data)

        # Assign back
        self.inner_graph = inner_graph
        self.outer_graph = outer_graph


    # --------------------------
    # Node Operations
    # --------------------------
    def add_node(self, node_data):
        self.nodes_data.append(node_data)
        self.create_hierarchical_graphs_iterative()

    def edit_node(self, label, new_data):
        for node in self.nodes_data:
            if node['label'] == label:
                node.update(new_data)
                break
        self.create_hierarchical_graphs_iterative()

    def delete_node(self, label):
        self.nodes_data = [node for node in self.nodes_data if node['label'] != label]
        self.edges_data = [edge for edge in self.edges_data if edge['start'] != label and edge['end'] != label]
        self.create_hierarchical_graphs_iterative()

    # --------------------------
    # Edge Operations
    # --------------------------
    def add_edges(self, edge_data):
        """Add one or more edges."""
        if isinstance(edge_data, list):
            self.edges_data.extend(edge_data)
        else:
            self.edges_data.append(edge_data)
        self.create_hierarchical_graphs_iterative()

    def edit_edges(self, start_end_pairs, new_data):
        """
        Edit one or more edges.
        `start_end_pairs`: tuple or list of tuples like [(start, end), ...]
        `new_data`: dict with new edge attributes
        """
        if isinstance(start_end_pairs, tuple):
            start_end_pairs = [start_end_pairs]

        for start, end in start_end_pairs:
            for edge in self.edges_data:
                if edge['start'] == start and edge['end'] == end:
                    edge.update(new_data)
                    break
        self.create_hierarchical_graphs_iterative()

    def delete_edges(self, start_end_pairs):
        """
        Delete one or more edges.
        `start_end_pairs`: tuple or list of tuples like [(start, end), ...]
        """
        if isinstance(start_end_pairs, tuple):
            start_end_pairs = [start_end_pairs]

        self.edges_data = [
            edge for edge in self.edges_data
            if (edge['start'], edge['end']) not in start_end_pairs
        ]
        self.create_hierarchical_graphs_iterative()


    # --------------------------
    # Visualization Methods
    # --------------------------
    def visualize_outer_graph(self, filename='outer_level_graph'):
        dot = graphviz.Digraph(comment='Outer Level Graph', engine='dot')

        # Add nodes with group-based fill color
        for node in self.outer_graph.nodes:
            fillcolor = self.group_colors.get(node, 'white')
            dot.node(str(node), style='filled', fillcolor=fillcolor)

        # Add directed edges with attributes
        for u, v, key, data in self.outer_graph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            color = data.get('color', 'black')
            dot.edge(str(u), str(v),
                    label=label,
                    penwidth=penwidth,
                    color=color)

        filepath = dot.render(filename, format='png', cleanup=True)
        print(f"Outer-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")

    def visualize_inner_graph_with_clusters(self, filename="inner_level_graph"):
        dot_source = 'digraph G {\n'
        dot_source += '    rankdir="LR";\n'
        dot_source += '    node [shape=box, style="filled"];\n'

        # Group nodes by their cluster (group)
        clusters = {}
        node_attrs = {}
        for node, attrs in self.inner_graph.nodes(data=True):
            group = attrs.get('group', 'DefaultGroup')
            clusters.setdefault(group, []).append(node)
            node_attrs[node] = attrs

        for group, nodes in clusters.items():
            bg_color = self.group_colors.get(group, 'white')
            cluster_id = group.replace(" ", "_")

            dot_source += f'    subgraph cluster_{cluster_id} {{\n'
            dot_source += f'        label="{group}";\n'
            dot_source += f'        bgcolor="{bg_color}";\n'

            for node in nodes:
                # Fallback to group color if no individual color provided
                node_color = node_attrs[node].get('color', self.group_colors.get(group, 'white'))
                dot_source += f'        "{node}" [fillcolor="{node_color}"];\n'

            dot_source += '    }\n'

        # Add edges
        for u, v, key, data in self.inner_graph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            color = data.get('color', 'black')
            dot_source += f'    "{u}" -> "{v}" [label="{label}", id="{key}", penwidth={penwidth}, color="{color}"];\n'

        dot_source += '}\n'

        dot = graphviz.Source(dot_source, format='png')
        filepath = dot.render(filename, cleanup=True)
        print(f"Inner-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")

    def visualize_subgraph(self, node_labels, filename="subgraph"):
        """
        Visualize a subgraph induced by a list of node labels.
        Nodes that don't exist are silently ignored.
        Parameters:
            node_labels (list of str): Nodes to include in the subgraph
            filename (str): Filename for output image
        """
        if not node_labels:
            print("No nodes provided for subgraph.")
            return

        # Filter only nodes that exist in the graph
        valid_nodes = [n for n in node_labels if n in self.inner_graph.nodes]
        if not valid_nodes:
            print("None of the provided nodes exist in the inner graph.")
            return

        # Create subgraph
        subgraph = self.inner_graph.subgraph(valid_nodes).copy()

        dot_source = 'digraph G {\n'
        dot_source += '    rankdir="LR";\n'
        dot_source += '    node [shape=box, style="filled"];\n'

        # Add nodes with colors
        for node in subgraph.nodes:
            attrs = subgraph.nodes[node]
            group = attrs.get('group', 'DefaultGroup')
            fillcolor = attrs.get('color', self.group_colors.get(group, 'white'))
            dot_source += f'    "{node}" [fillcolor="{fillcolor}"];\n'

        # Add edges with styling
        for u, v, key, data in subgraph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            color = data.get('color', 'black')
            dot_source += f'    "{u}" -> "{v}" [label="{label}", id="{key}", penwidth={penwidth}, color="{color}"];\n'

        dot_source += '}\n'

        dot = graphviz.Source(dot_source, format='png')
        filepath = dot.render(filename, cleanup=True)
        print(f"Subgraph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the subgraph.")


    #----Get Node, edge attributes

    def get_node_attributes(self, labels=None):
        """
        Get attributes of one or more nodes.
        If labels is None or [] → return ALL node attributes.
        Parameters:
            labels (str or list of str or None)
        Returns:
            dict: {label: attributes}
        """

        # return ALL
        if labels is None or labels == []:
            return {label: dict(attrs) for label, attrs in self.inner_graph.nodes(data=True)}

        # normalize input
        if isinstance(labels, str):
            labels = [labels]

        results = {}
        for label in labels:
            if label in self.inner_graph.nodes:
                results[label] = dict(self.inner_graph.nodes[label])
            else:
                results[label] = None
        return results


    def get_edge_attributes(self, edge_tuples=None):
        """
        Get attributes of one or more edges.
        If edge_tuples is None or [] → return ALL edges in the graph
        Parameters:
            edge_tuples: tuple or list of tuples [(start,end),...]
        Returns:
            dict: {(start, end, key): attributes}
        """

        results = {}

        # return ALL edges
        if edge_tuples is None or edge_tuples == []:
            for u, v, key, attrs in self.inner_graph.edges(keys=True, data=True):
                results[(u, v, key)] = dict(attrs)
            return results

        # normalize input
        if isinstance(edge_tuples, tuple):
            edge_tuples = [edge_tuples]

        # specific subset edges
        for start, end in edge_tuples:
            if self.inner_graph.has_edge(start, end):
                for key, attrs in self.inner_graph[start][end].items():
                    results[(start, end, key)] = dict(attrs)
            else:
                results[(start, end, None)] = None
        return results


In [ ]:
nodes = [
    {'label': 'A', 'group': 'Group A', 'type': 'system', 'color':'yellow','description':'This node does this'},
    {'label': 'B', 'group': 'Group A', 'type': 'user','color':'yellow','description':'This node does this'},
    {'label': 'C', 'group': 'Group B', 'type': 'system','color':'orange','description':'This node does B'},
    {'label': 'D', 'group': 'Group B', 'type': 'user','color':'orange','description':'This node does B'},
    {'label': 'E', 'group': 'Group C', 'type': 'system','color':'pink','description':'This node does c'},
]

edges = [
    {'start': 'A', 'end': 'B', 'type': 'type1', 'weight': 0.4, 'color':'blue','description':'This edge is type-1'},
    {'start': 'A', 'end': 'C', 'type': 'type2', 'weight': 0.8,'description':'This edge is type-2'},
    {'start': 'B', 'end': 'C', 'type': 'type1', 'weight': 0.5,'description':'This edge is type-1'},
    {'start': 'C', 'end': 'E', 'type': 'type3', 'weight': 0.1,'description':'This edge is type-3'},
    {'start': 'C', 'end': 'E', 'type': 'type1', 'weight': 0.9}, # Multi-edge here
    {'start': 'D', 'end': 'E', 'type': 'type2', 'weight': 0.3},
    {'start': 'E', 'end': 'D', 'type': 'type4', 'weight': 0.3},
]

In [ ]:
hg = HierarchicalGraph(nodes, edges)
hg.visualize_inner_graph_with_clusters()
hg.visualize_outer_graph()

In [ ]:
hg.get_node_attributes(labels=["A","C"])

In [ ]:
hg.get_edge_attributes(edge_tuples=[('A','B'),('C','E')])

In [ ]:
hg.add_node({'label': 'F', 'group': 'Group A', 'type': 'user'})

In [ ]:
hg.add_edges(edge_data=[
    {'start': 'C', 'end': 'A', 'type': 'type5', 'weight': 0.4},
    {'start': 'F', 'end': 'E', 'type': 'type5', 'weight': 0.4}
])

In [ ]:
hg.visualize_inner_graph_with_clusters()

In [ ]:
hg.visualize_subgraph(["A","B","E"])

#Maintenance Management

## References

### Scientific Literature using Graph / Knowledge Graph Approaches in Industrial Maintenance

1. **Xia et al. (2023)**  
   *Maintenance planning recommendation of complex industrial equipment based on knowledge graph and graph neural network*  
   [Reliability Engineering & System Safety, Vol 232](https://doi.org/10.1016/j.ress.2022.109068)  
   DOI: 10.1016/j.ress.2022.109068

2. **Lou et al. (2023)**  
   *Knowledge Graph Construction Based on a Joint Model for Equipment Maintenance*  
   [Mathematics, 11(17): 3748](https://www.mdpi.com/2227-7390/11/17/3748)  
   DOI: 10.3390/math11173748

3. **Teern et al. (2022)**  
   *Knowledge graph construction and maintenance process: Design challenges for industrial maintenance support*  
   [CEUR Workshop Proceedings (PDF)](https://www.researchgate.net/publication/363926032_Knowledge_graph_construction_and_maintenance_process_Design_challenges_for_industrial_maintenance_support)

4. **Stewart et al. (2024)**  
   *MWO2KG and Echidna: Constructing and exploring an interactive maintenance knowledge graph*  
   [Journal of Maintenance & Innovation (DOI)](https://journals.sagepub.com/doi/10.1177/1748006X221131128)

5. **Cai et al. (2024)**  
   *Knowledge graph‑driven equipment fault diagnosis method for intelligent manufacturing*  
   [Int J Adv Manufacturing Technology, Vol 130](https://link.springer.com/article/10.1007/s00170-024-12998-x)

6. **Pérez Hernández (2022)**  
   *Maintenance Strategies for Networked Assets*  
   [University of Cambridge Repository (PDF)](https://www.repository.cam.ac.uk/bitstream/handle/1810/336867/Maintenance_Strategies_for_Networked_Assets.pdf)

7. **Barberá et al. (2013)**  
   *The Graphical Analysis for Maintenance Management Method (GAMM)*  
   [ResearchGate (PDF)](https://www.researchgate.net/publication/262906105_The_Graphical_Analysis_for_Maintenance_Management_Method_A_Quantitative_Graphical_Analysis_to_Support_Maintenance_Management_Decision_Making)

8. **Ammann et al. (2025)**  
   *Automated Knowledge Graph Learning in Industrial Processes*  
   [TRINEFLEX Project Report (PDF)](https://trineflex.eu/wp-content/uploads/2025/08/Automated-Knowledge-Graph-Learning-Lolita-Ammann.pdf)

9. **Zheng et al. (2022)**  
   *Query-based Industrial Analytics over Knowledge Graphs with Ontology Reshaping*  
   [arXiv preprint](https://arxiv.org/abs/2209.11089)

10. **Fenza et al. (2020)**  
    *A Cognitive Approach based on the Actionable Knowledge Graph for supporting Maintenance Operations*  
    [arXiv preprint](https://arxiv.org/abs/2011.09554)
